In [1]:
#INSTALLATIONS

#On first run, installation of the non-default package below is required
#pip install beautifulsoup4

In [1]:
#PACKAGE IMPORT

#To send data request to website url
import requests as rqs

#To interpret website url html code
from bs4 import BeautifulSoup

#To format output for visualisation checks
import pandas as pd

#To stagger request intervals to allow unhindered functioning of the host website
import random
import time


In [2]:
#WEB SCRAPING
#INPUT: GENRE + NUMBER OF PAGES

#Inspect -> Network -> refresh -> select top -> copy request headers
#required information from html script for each request batch
#COOKIE INFORMATION REMOVED FOR PRIVACY BUT MUST BE SUPPLIED FOR FUNCTION
headers = {
    'Accept' : 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8',
    'Accept-Encoding' : 'gzip, deflate, br',
    'Accept-Language' : 'en-GB,en;q=0.5',
    'Connection' : 'keep-alive',
    'Cookie' : '', #REMOVED INFO
    'Host' : 'www.metacritic.com',
    'Sec-Fetch-Dest' : 'document',
    'Sec-Fetch-Mode' : 'navigate',
    'Sec-Fetch-Site' : 'none',
    'Sec-Fetch-User' : '?1',
    'TE' : 'trailers',
    'Upgrade-Insecure-Requests' : '1',
    'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:109.0) Gecko/20100101 Firefox/116.0',
}

#Set Genre Info to current page category
genre = 'flight'

#Set number of pages to check through
pages = list(map(str, range(3)))

#lists and empty dictionary to store current and running page scrape
page_dict = {}
position_list = []
title_list = []
platform_list = []
pub_date_list = []
rating_list = []

#go through number of pages supplied
for page in pages:
    
    #Excecute each url request on intervals between 35 to 108 seconds.
    time.sleep(random.randint(35,108)) 
    
    #print message to indicate progression by page number on iteration
    print(f'starting page{page}')
    
    #request url based on supplied genre, current page iteration and supplied headers
    data = rqs.get(f'https://www.metacritic.com/browse/games/genre/metascore/{genre}/all?view=condensed&page={page}', headers=headers)
    
    #store scraped html script to a variable
    ratehtml = BeautifulSoup(data.text, 'html.parser')
    
    #Note: td class="details" contains each entry segment in the page
    #Select the start of each repeating segment of the webpage by repeating object tr.expand_collapse
    rateentry = ratehtml.select('tr.expand_collapse')
    
    #select relative html location, retrieve info as text and add it to appropriate list
    #Iterate through each repeating segment found to retrieve information and append to item specific lists
    for entry in rateentry:
        
        #Retrieve the rating rank position
        position = entry.select('td.details a span.title.numbered')[0].get_text()
        #Strip the white space and /n and remove full stops and convert to integers
        position_list.append(int(position.strip()[0:-1]))
                        
        #Retrieve the title of the game
        title = entry.select('td.details h3')[0].get_text()
        title_list.append(title)
        
        #Retrieve the platform the game is released and rated on as Stripped characters only
        platform = entry.select('td.details div.platform span.data')[0].get_text()
        platform_list.append(platform.strip())
        
        #Retrieve the date of the game release
        pub_date = entry.select('td.details span')[3].get_text()
        pub_date_list.append(pub_date)
    
        #Retrieve the metacritic rating as an integer
        rating = entry.select('td.score a.metascore_anchor div')[0].get_text()
        rating_list.append(int(rating))

#Once interation through each entry of each page is complete store the appended list output in the dictionary
page_dict = {"position": position_list, "title": title_list, "platform": platform_list, "pub_date": pub_date_list, "rating": rating_list}
    
#convert to a dataframe to clean up
mc_rating = pd.DataFrame.from_dict(page_dict)

#label with entries with the specified genre
mc_rating['genre'] = genre

#convert date format for use (not retained in csv)
mc_rating['pub_date'] = pd.to_datetime(mc_rating['pub_date'])

#view the dataframe
mc_rating


starting page0
starting page1
starting page2
starting page3
starting page4
starting page5
starting page6
starting page7
starting page8
starting page9
starting page10
starting page11
starting page12
starting page13
starting page14
starting page15
starting page16
starting page17
starting page18
starting page19
starting page20
starting page21
starting page22
starting page23
starting page24
starting page25
starting page26
starting page27
starting page28
starting page29
starting page30
starting page31
starting page32
starting page33
starting page34
starting page35
starting page36
starting page37
starting page38
starting page39
starting page40
starting page41
starting page42
starting page43
starting page44
starting page45
starting page46
starting page47
starting page48
starting page49
starting page50
starting page51
starting page52
starting page53
starting page54
starting page55
starting page56
starting page57
starting page58
starting page59
starting page60
starting page61
starting page62
st

,position,title,platform,pub_date,rating,genre
0,1,SoulCalibur,Dreamcast,1999-09-08,98,action
1,2,Super Mario Galaxy,Wii,2007-11-12,97,action
2,3,Super Mario Galaxy 2,Wii,2010-05-23,97,action
3,4,Perfect Dark,Nintendo 64,2000-05-22,97,action
4,5,Metroid Prime,GameCube,2002-11-17,97,action
...,...,...,...,...,...,...
11919,11920,Double Dragon II: Wander of the Dragons,Xbox 360,2013-04-05,17,action
11920,11921,Motorbike,PlayStation 3,2013-06-25,17,action
11921,11922,101-in-1 Explosive Megamix,Wii,2011-10-13,16,action
11922,11923,Waifu Impact,Switch,2022-04-28,15,action


In [3]:
#EXPORTING

#export each batch result dataframe as a csv named based on supplied genre information
mc_rating.to_csv(f'{genre}_MetaCritScores.csv')

In [ ]:
#EXTRA INFORMATION

#POSSIBLE UPDATE METHOD:

#Iterate through all the genres and take the most recent entry page for each.
#Check for title overlap with current database else take additional pages until match.
#Remove entries that match the database and then add the newer entries to the database.

#PURPOSE OF SCRIPT:

#Metacritic entry display has since been updated so scraping in the same manner is no longer possible.
#The above process is only for documentation of how the MetaCriticScores.csv was created.


